In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/tmp/ipykernel_39144/912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## RR interval 사용방법

1. [S1-S1-Phonocardiogram](https://github.com/kevinmgamboa/S1-S1-Phonocardiogram-Peak-Detection-Method-in-Python) git clone 진행
2. [PeakUtils](https://bitbucket.org/lucashnegri/peakutils/src/master/) git clone 진행후 **python setup.py install**  


* RR interval feature 추출할 때 오류가 발생한다면, Filters1.mat 파일 복사해서 현재 notebook 파일이 있는 위치에 Filters1.mat 이름으로 위치시키기

In [4]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt                 # Librery to load .mat files
import peakutils                                # Librery to help in peak detection
from scipy import special

##----------------------------------------------------------------------------
# FUNCTIONS: "execfile"
##----------------------------------------------------------------------------
''' This Function creates a time-vector for any signal given the sampling frequency
    and the duration of a signal'''
def time_vector(sampling_frequency,duration): 
	number_samples= sampling_frequency*duration;
	result=np.arange(1,duration/number_samples,duration-duration/number_samples);

	return result

##----------------------------------------------------------------------------
''' Derivate of an input signal as y[n]= x[n+1]- x[n-1] 
'''
def derivate (x):
	lenght=np.shape(x)				# Get the length of the vector		
	lenght=lenght[0];				# Get the value of the length
	y=np.zeros(lenght);				# Initializate derivate vector
	for i in range(lenght-1):
			y[i]=x[i+1]-x[i];		
	return y

##----------------------------------------------------------------------------
'''To normalized any vector\0-dimentional array in [-1,1] range, by divided the 
   vector by the maximun value of itself, substracting the mean value to the vector
   & dividing each value of the vector by the maximun value of itself 
'''
def vec_nor(x):
	lenght=np.shape(x)				# Get the length of the vector		
	lenght=lenght[0];				# Get the value of the length
	xMax=max(x);					# Get the maximun value of the vector
	nVec=np.zeros(lenght);			        # Initializate derivate vector
	for n in range(lenght):
		nVec[n]=x[n]/xMax;			
	nVec=nVec-np.mean(nVec);
	nVec=np.divide(nVec,np.max(nVec));
	return nVec
##----------------------------------------------------------------------------
'''
  FpassBand is the function that develop a pass band filter of the signal 'x' through the
  discrete convolution of this 'x' first with the coeficients of a High Pass Filter 'hp' and then
  with the discrete convolution of this result with a Low Pass Filter 'lp'
'''
def FpassBand(X,hp,lp):
        llp=np.shape(lp)	  	        # Get the length of the lowpass vector		
        llp=llp[0];				# Get the value of the length
        lhp=np.shape(hp)			# Get the length of the highpass vector		
        lhp=lhp[0];				# Get the value of the length	

        x=np.convolve(X,lp);		        # Disrete convolution 
        x=x[round(llp/2):round(-1-(llp/2))];
        x=x-(np.mean(x));
        x=x/np.max(x);
	
        y=np.convolve(x,hp);			# Disrete onvolution
        y=y[round((lhp/2)):round(-1-(lhp/2))];
        y=y-np.mean(y);
        y=y/np.max(y);

        x=np.convolve(y,lp);		        # Disrete convolution 
        x=x[round((llp/2)):round(-1-(llp/2))];
        x=x-(np.mean(x));
        x=x/np.max(x);
	
        y=np.convolve(x,hp);			# Disrete onvolution
        y=y[round((lhp/2)):round(-1-(lhp/2))];
        y=y-np.mean(y);
        y=y/np.max(y);
        
        y=vec_nor(y);				# Vector Normalizing
        
        return y

## LSTM_fix



In [2]:
pwd

'/Data2/hmd/hmd_sy/notebooks'

In [3]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display
import math
import sys

sys.path.insert(0,'/Data2/hmd/hmd_sy/evaluation-2022')


## RR interval
sys.path.insert(0,'peakutils')
sys.path.insert(0,'S1-S1-Phonocardiogram-Peak-Detection-Method-in-Python')
import peakutils
sys.path.insert(0,'utils')

from helper_code import *
from get_feature import *
from models import *
from Generator0 import *
import tensorflow as tf
import datetime
from evaluate_model import *
from scipy import special
import scipy.io as sio

import tensorflow as tf
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [5]:
pwd

'/Data2/hmd/hmd_sy/notebooks'

In [6]:

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Fri Aug 12 02:08:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:1A:00.0 Off |                  Off |
| 35%   60C    P2   192W / 260W |  12944MiB / 48601MiB |     67%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 8000     On   | 00000000:3D:00.0 Off |                  Off |
| 48%   

In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[3], 'GPU')
    except RuntimeError as e:
        print(e)

In [8]:
data_folder =  '/Data2/hmd/physionet.org/files/circor-heart-sound/1.0.3/training_data'
train_folder =  '/Data2/hmd/data_split/murmur/train'
test_folder = '/Data2/hmd/data_split/murmur/test'

## wav2vec2

In [9]:
import os

import tensorflow as tf
import tensorflow_hub as hub
from wav2vec2 import Wav2Vec2Config

model_url = "https://tfhub.dev/vasudevgupta7/wav2vec2-960h/1"
pretrained_layer = hub.KerasLayer(model_url, trainable=True)

inputs = tf.keras.Input(shape=(246000,))
hidden_states = pretrained_layer(inputs)
outputs = tf.keras.layers.Dense(10)(hidden_states)

wav2vec2_model = tf.keras.Model(inputs=inputs, outputs=outputs)
wav2vec2_model.summary()

@tf.function(jit_compile=True)
def forward(speech):
    return wav2vec2_model(speech, training=True)

2022-08-12 02:09:07.733529: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-12 02:09:08.454015: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 47232 MB memory:  -> device: 3, name: Quadro RTX 8000, pci bus id: 0000:40:00.0, compute capability: 7.5


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 246000)]          0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 768, 32)           94396320  
_________________________________________________________________
dense (Dense)                (None, 768, 10)           330       
Total params: 94,396,650
Trainable params: 94,396,650
Non-trainable params: 0
_________________________________________________________________


In [10]:
def sigmoidal_decay(e, start=0, end=100, lr_start=1e-3, lr_end=1e-5):
    if e < start:
        return lr_start
    elif e > end:
        return lr_end

    middle = (start + end) / 2
    s = lambda x: 1 / (1 + np.exp(-x))

    return s(13 * (-e + middle) / np.abs(end - start)) * np.abs(lr_start - lr_end) + lr_end

In [11]:
# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

#    @tf.function
    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')
        


In [12]:
import pickle as pk
def save_challenge_model(model_folder, model1, model2, m_name1, m_name2, param_feature) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename1 = os.path.join(model_folder, m_name1 + '_model1.hdf5')
    filename2 = os.path.join(model_folder, m_name2 + '_model2.hdf5')
    model1.save(filename1)
    model2.save(filename2)
    param_feature['model1'] = m_name1
    param_feature['model2'] = m_name2
    param_feature['model_fnm1'] = filename1
    param_feature['model_fnm2'] = filename2
    with open(info_fnm, 'wb') as f:
        pk.dump(param_feature, f, pk.HIGHEST_PROTOCOL)
    return 1

def load_challenge_model(model_folder, verbose):
    info_fnm = os.path.join(model_folder, 'desc.pk')
    with open(info_fnm, 'rb') as f:
        info_m = pk.load(f)
#    if info_m['model'] == 'toy' :
#        model = get_toy(info_m['mel_shape'])
#    filename = os.path.join(model_folder, info_m['model'] + '_model.hdf5')
#    model.load_weights(filename)
    return info_m

In [13]:
def run_challenge_model(model, data, recordings, verbose):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']
    
    if model['model1'] == 'lcnn1_dr' :
        model1 = get_LCNN_o_4_dr_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], 
                                 model['interval_input_shape'],model['wav2_input_shape'],use_wav2=model['use_wav2'],
                                 use_mel = model['use_mel'],use_cqt = model['use_cqt'], use_stft = model['use_stft'], 
                                 ord1 = model['ord1'], dp = model['dp'], fc = model['fc'], ext = model['ext'])
    if model['model2'] == 'lcnn2_dr' :
        model2 = get_LCNN_o_4_dr_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], 
                               model['interval_input_shape'],model['wav2_input_shape'],use_wav2=model['use_wav2'],
                               use_mel = model['use_mel'], use_cqt = model['use_cqt'], use_stft = model['use_stft'], 
                               dp = model['dp'], fc = model['fc'], ext = model['ext'])
    model1.load_weights(model['model_fnm1'])
    model2.load_weights(model['model_fnm2'])
    

    
    maxlen1 = params_feature['maxlen1']
    min_dist = params_feature['min_dist']
    max_interval_len = params_feature['max_interval_len']
    
    
    # Load features.
    features = get_feature_one(data, verbose = 0)

    samp_sec = model['samp_sec'] 
    pre_emphasis = model['pre_emphasis']
    hop_length = model['hop_length']
    win_length = model['win_length']
    n_mels = model['n_mels']
    filter_scale = model['filter_scale']
    n_bins = model['n_bins']
    fmin = model['fmin']
    use_mel = model['use_mel']
    use_cqt = model['use_cqt']
    use_stft = model['use_stft']
#    use_raw = model['use_raw']
    trim = model['trim']
    
    # wav2vec2
    use_wav2 = model['use_wav2']
    maxlen1 = model['maxlen1']
    features['wav2'] = []
    tmp_wav=[] 
    tmp = []
    
    # RR interval
    use_interval = model['use_interval']
    min_dist = model['min_dist']
    max_interval_len = model['max_interval_len']
    per_sec = model['per_sec']

    features['interval'] = []
    tmp_total_interval = [] 
    
    if use_wav2:

        for i in range(len(recordings)):
            data = recordings[i]
            tmp_wav.append(data)

        padded =pad_sequences(tmp_wav, maxlen=maxlen1, dtype='float64', padding='post', truncating='post', value=0.0)

        padded=np.array(padded, dtype=np.float32)
        for i in range(len(padded)):
            features['wav2'].append(padded[i])

        features['wav2'] = np.array(features['wav2'])
    

       
    if use_interval:


        for i in tqdm.tqdm(range(len(recordings))) :
            
            datos=recordings[i]
            filtros=sio.loadmat('./Filters1')
            tmp_interval = []
            n_samp = len(datos)//per_sec


            try:
                for k in range(n_samp):
                    
                    X = datos[k*per_sec:(k+1)*per_sec]
                    Fs= per_sec
                    Fpa20=filtros['Fpa20'];			        # High pass filter
                    Fpa20=Fpa20[0];					# High pass filter
                    Fpb100=filtros['Fpb100'];		        # Low-pass Filter
                    Fpb100=Fpb100[0];				# Low-pass Filter
                    Xf=FpassBand(X,Fpa20,Fpb100); 	                # Apply a passband filter
                    Xf=vec_nor(Xf);			

                # Derivate of the Signal
                    dX=derivate(Xf);				# Derivate of the signal
                    dX=vec_nor(dX);					# Vector Normalizing
                # Square of the signal
                    dy=np.square(Xf);
                    dy=vec_nor(dy);

                    size=np.shape(Xf)				# Rank or dimension of the array
                    fil=size[0];					# Number of rows

                    positive=np.zeros((1,fil+1));                   # Initializating Positives Values Vector 
                    positive=positive[0];                           # Getting the Vector

                    points=np.zeros((1,fil));                       # Initializating the all Peak Points Vector
                    points=points[0];                               # Getting the point vector

                    peaks=np.zeros((1,fil));                        # Initializating the s1-s1 Peak Vector
                    peaks=peaks[0];                                 # Getting the point vector


                    for i in range(0,fil):
                        if dX[i]>0:
                            positive[i]=1;
                        else:
                            positive[i]=0;

                    for i in range(0,fil):
                        if (positive[i]==1 and positive[i+1]==0):
                            points[i]=Xf[i];
                        else:
                            points[i]=0;

                    indexes=peakutils.indexes(points,thres=0.5/max(points), min_dist=min_dist);
                    lenght=np.shape(indexes)			# Get the length of the index vector		
                    lenght=lenght[0];				# Get the value of the index vector

                    for i in range(0,lenght):
                        p=indexes[i];
                        peaks[p]=points[p];

                    n=np.arange(0,fil);                            # Vector to the X axes (Number of Samples)
                    indexes =indexes+(k*per_sec)    
                    tmp_peaks = np.array(indexes)


                    tmp_interval.extend(tmp_peaks)

                tmp_interval = np.array(tmp_interval)
                tmp_interval = np.diff(tmp_interval)

                tmp_total_interval.append(tmp_interval)


            except:
                print(i)
                tmp_peaks = np.zeros(max_interval_len)
                tmp_total_interval.append(tmp_peaks)

    else :

        tmp_peaks = np.zeros(max_interval_len)
        tmp_total_interval.append(tmp_peaks)       


    if use_interval:
          
        padded =pad_sequences(tmp_total_interval, maxlen=max_interval_len, dtype='float64', padding='post', truncating='post', value=0.0)

        for i in range(len(padded)):
            features['interval'].append(padded[i])
             
        for i in range(len(features['interval'])):
            features['interval'][i]= features['interval'][i].reshape(-1,1)
    
        features['interval']=np.array(features['interval'])
    
        
    else:
        for i in range(len(tmp_interval)):
            features['interval'].append(tmp_total_interval[i])
        
        for i in range(len(features['interval'])):
            features['interval'][i]= features['interval'][i].reshape(-1,1)
        features['interval']=np.array(features['interval'])
        
        
    
    features['mel1'] = []
    for i in range(len(recordings)) :
        if use_mel :
            mel1 = feature_extract_melspec(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                           win_length = win_length, n_mels = n_mels, trim = trim)[0]
        else :
            mel1 = np.zeros( (1,1) )
        features['mel1'].append(mel1)
    M, N = features['mel1'][0].shape

    if use_mel :
        for i in range(len(features['mel1'])) :
            features['mel1'][i] = features['mel1'][i].reshape(M,N,1)
    features['mel1'] = np.array(features['mel1'])

    features['cqt1'] = []
    for i in range(len(recordings)) :
        if use_cqt :
            mel1 = feature_extract_cqt(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale, 
                                        n_bins = n_bins, fmin = fmin, trim = trim)[0]
        else :
            mel1 = np.zeros( (1,1))
        features['cqt1'].append(mel1)
    M, N = features['cqt1'][0].shape
    if use_cqt :
        for i in range(len(features['cqt1'])) :
            features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)   
    features['cqt1'] = np.array(features['cqt1'])

    features['stft1'] = []
    for i in range(len(recordings)) :
        if use_stft :
            mel1 = feature_extract_stft(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                        win_length = win_length, trim = trim)[0]
        else :
            mel1 = np.zeros( (1,1) )
        features['stft1'].append(mel1)
    M, N = features['stft1'][0].shape
    if use_stft :
        for i in range(len(features['stft1'])) :
            features['stft1'][i] = features['stft1'][i].reshape(M,N,1)           
    features['stft1'] = np.array(features['stft1'])

    #    print(features)
    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], 
#                            features['interval'],
                           features['interval'],features['wav2'],
                           features['mel1'], features['cqt1'], features['stft1']])
    
    res2 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], 
#                            features['interval'],
                           features['interval'],features['wav2'],
                           features['mel1'], features['cqt1'], features['stft1']])

    # Get classifier probabilities.
    if model['ord1'] :
        idx1 = res1.argmax(axis=0)[0]
        murmur_p = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
        murmur_probabilities = np.zeros((3,))
        murmur_probabilities[0] = murmur_p[0]
        murmur_probabilities[1] = 0
        murmur_probabilities[2] = murmur_p[1]
        outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기
    else :
        if model['mm_mean'] :
            murmur_probabilities = res1.mean(axis = 0)
        else :
            idx1 = res1.argmax(axis=0)[0]
            murmur_probabilities = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
        outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기

    # Get classifier probabilities.
#    idx1 = res1.argmax(axis=0)[0]
#    murmur_probabilities = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
#    outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기
#    idx = np.argmax(prob1)

    ## 이부분도 생각 필요.. rule 을 cost를 maximize 하는 기준으로 threshold 탐색 필요할지도..
    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
#    idx = np.argmax(murmur_probabilities)
    if murmur_probabilities[0] > 0.482 :
        idx = 0
    else :
        idx = 2
    murmur_labels[idx] = 1

    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    if outcome_probabilities[0] > 0.607 :
        idx = 0
    else :
        idx = 1    
        # idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1
    
    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))
    
    return classes, labels, probabilities


In [14]:

patient_files_trn = find_patient_files(train_folder)
patient_files_test = find_patient_files(test_folder)

### 수정할 부분

In [15]:
model_folder = 'lcnn_fix_rr_w2v2'
output_folder = '/Data2/hmd/hmd_sy/notebooks/out_lcnn_fix_rr_w2v2'

In [16]:
per_sec=16000 # 4초
#-------------------------
winlen = 512
hoplen = 256
nmel = 120 #np.random.choice([100, 120, 140])
nsec = 20 #np.random.choice([10, 20, 30, 40, 50])
trim = 0 #np.random.choice([0,2000, 4000])
use_mel = True
use_cqt = False #np.random.choice([True,False])
use_stft = False#np.random.choice([True, False])
use_raw=False
use_interval=True
use_wav2=True
maxlen1 = 246000
min_dist = 500
max_interval_len = 192

params_feature = {'samp_sec': nsec,
              #### melspec, stft 피쳐 옵션들  
              'pre_emphasis': 0,
              'hop_length': hoplen,
              'win_length':winlen,
              'n_mels': nmel,
              #### cqt 피쳐 옵션들  
              'filter_scale': 1,
              'n_bins': 80,
              'fmin': 10,
              ### RR interval 옵션
                  'min_dist':min_dist,
                  'max_interval_len' : max_interval_len,
                  'per_sec' : per_sec,
                  'use_interval' : use_interval,
            ### wav2vec2 옵션
                  'maxlen1': maxlen1,
                  'use_wav2' : use_wav2,
              ### 사용할 피쳐 지정
                  'trim' : trim, # 앞뒤 얼마나 자를지? 4000 이면 1초
                  'use_mel' : use_mel,
                  'use_cqt' : use_cqt,
                  'use_stft' : use_stft          
}


mm_weight = 3 #np.random.choice([2,3,4,5])
oo_weight = 3 #np.random.choice([2,3,4,5,6])
ord1 = True #np.random.choice([True,False])
mm_mean = False #np.random.choice([True,False])
dp = 0 #np.random.choice([0, .1, .2, .3])
fc = False #np.random.choice([True,False])
ext = False
chaug = 10 #np.random.choice([0, 10])
mixup = True #np.random.choice([True,False])
cout = .8 #np.random.choice([0, 0.8])
wunknown = 1 #np.random.choice([1, 0.7, .5, .2])
n1 = 0 #np.random.choice([0,2])
if n1 == 0 :
    ranfil = False
else :
    ranfil = [n1, [18,19,20,21,22,23]]

In [17]:
def get_features_3lb_all_ord(data_folder, patient_files_trn, po = .3,
                          samp_sec=20, pre_emphasis = 0, hop_length=256, win_length = 512, n_mels = 100,
                             filter_scale = 1, n_bins = 80, fmin = 10, trim = 4000,
                             use_mel = True, use_cqt = False, use_stft = False, use_raw = False,
                             use_wav2=False, maxlen1=120000,
                             use_interval=False,min_dist=500,per_sec=16000,max_interval_len=192
                         ) :
    features = dict()
    features['id'] = []
    features['age'] = []
    features['sex'] = []
    features['hw'] = []
    features['preg'] = []
    features['loc'] = []
    features['mel1'] = []
    features['cqt1'] = []
    features['stft1'] = []
    features['raw1'] = []
    
    features['interval'] = []
    tmp_total_interval = [] 
    
    features['wav2']=[]
    tmp_wav=[]

    
    mm_labels = []
    out_labels = []

    age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

    num_patient_files = len(patient_files_trn)

    for i in tqdm.tqdm(range(num_patient_files)):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files_trn[i])
        num_locations = get_num_locations(current_patient_data)
        recording_information = current_patient_data.split('\n')[1:num_locations+1]
        for j in range(num_locations) :
            entries = recording_information[j].split(' ')
            recording_file = entries[2]
            filename = os.path.join(data_folder, recording_file)

            # Extract id
            id1 = recording_file.split('_')[0]
            features['id'].append(id1)

            # Extract melspec
            if use_mel :
                mel1 = feature_extract_melspec(filename, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                               win_length = win_length, n_mels = n_mels, trim = trim)[0]
            else :
                mel1 = np.zeros( (1,1,1) )
            features['mel1'].append(mel1)

            if use_cqt :
                mel2 = feature_extract_cqt(filename, samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale, 
                                           n_bins = n_bins, fmin = fmin, trim = trim)[0]
            else :
                mel2 = np.zeros( (1,1,1) )
            features['cqt1'].append(mel2)

            if use_stft :
                mel3 = feature_extract_stft(filename, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                            win_length = win_length, trim = trim)[0]
            else :
                mel3 = np.zeros( (1,1,1) )
            features['stft1'].append(mel3)

            if use_raw :
                frequency1, recording1 = sp.io.wavfile.read(filename)
            else :
                recording1 = np.zeros((1))
            features['raw1'].append(recording1)
            
            ## w2v2
            if use_wav2:
#                 frequency1, recording1 = sp.io.wavfile.read(filename)
                recording1,frequency1 = librosa.load(filename)
                
            else :
                recording1 = np.zeros( (1) )
            tmp_wav.append(recording1) 

            ## RR interval
            if use_interval :
                
                datos=sp.io.wavfile.read(filename)
                filtros=sio.loadmat('./Filters1')
                tmp_interval = []
                n_samp = len(datos[1])//per_sec
                               
                
                try:
                    for k in range(n_samp):
                        X = datos[1][k*per_sec:(k+1)*per_sec]
                        Fs= datos[0]
                        Fpa20=filtros['Fpa20'];			        # High pass filter
                        Fpa20=Fpa20[0];					# High pass filter
                        Fpb100=filtros['Fpb100'];		        # Low-pass Filter
                        Fpb100=Fpb100[0];				# Low-pass Filter
                        Xf=FpassBand(X,Fpa20,Fpb100); 	                # Apply a passband filter
                        Xf=vec_nor(Xf);			
            
            # Derivate of the Signal
                        dX=derivate(Xf);				# Derivate of the signal
                        dX=vec_nor(dX);					# Vector Normalizing
            # Square of the signal
                        dy=np.square(Xf);
                        dy=vec_nor(dy);
                    
                        size=np.shape(Xf)				# Rank or dimension of the array
                        fil=size[0];					# Number of rows

                        positive=np.zeros((1,fil+1));                   # Initializating Positives Values Vector 
                        positive=positive[0];                           # Getting the Vector

                        points=np.zeros((1,fil));                       # Initializating the all Peak Points Vector
                        points=points[0];                               # Getting the point vector

                        peaks=np.zeros((1,fil));                        # Initializating the s1-s1 Peak Vector
                        peaks=peaks[0];                                 # Getting the point vector

            
                        for i in range(0,fil):
                            if dX[i]>0:
                                positive[i]=1;
                            else:
                                positive[i]=0;
    
                        for i in range(0,fil):
                            if (positive[i]==1 and positive[i+1]==0):
                                points[i]=Xf[i];
                            else:
                                points[i]=0;

                        indexes=peakutils.indexes(points,thres=0.5/max(points), min_dist=min_dist);
                        lenght=np.shape(indexes)			# Get the length of the index vector		
                        lenght=lenght[0];				# Get the value of the index vector

                        for i in range(0,lenght):
                            p=indexes[i];
                            peaks[p]=points[p];
        
                        n=np.arange(0,fil);                            # Vector to the X axes (Number of Samples)
                        indexes =indexes+(k*per_sec)    
                        tmp_peaks = np.array(indexes)
                    
                    
                        tmp_interval.extend(tmp_peaks)
                    
                    tmp_interval = np.array(tmp_interval)
                    tmp_interval = np.diff(tmp_interval)
                    
                    tmp_total_interval.append(tmp_interval)
 
           
                except:
                    print(filename)
                    tmp_peaks = np.zeros(max_interval_len)
                    tmp_total_interval.append(tmp_peaks)
                    
            else :
                        
                tmp_peaks = np.zeros(max_interval_len)
                tmp_total_interval.append(tmp_peaks)               
            
            
            
            # Extract age_group
            age_group = get_age(current_patient_data)
            current_age_group = np.zeros(6, dtype=int)
            if age_group in age_classes:
                j = age_classes.index(age_group)
                current_age_group[j] = 1
            else :
                current_age_group[5] = 1
            features['age'].append(current_age_group)

            # Extract sex
            sex = get_sex(current_patient_data)
            sex_features = np.zeros(2, dtype=int)
            if compare_strings(sex, 'Female'):
                sex_features[0] = 1
            elif compare_strings(sex, 'Male'):
                sex_features[1] = 1
            features['sex'].append(sex_features)

            # Extract height and weight.
            height = get_height(current_patient_data)
            weight = get_weight(current_patient_data)
            ## simple impute
            if math.isnan(height) :
                height = 110.846
            if math.isnan(weight) :
                weight = 23.767
                
            features['hw'].append(np.array([height, weight]))

            # Extract pregnancy
            is_pregnant = get_pregnancy_status(current_patient_data)
            features['preg'].append(is_pregnant)

            # Extract location
            locations = entries[0]
            num_recording_locations = len(recording_locations)
            loc_features = np.zeros(num_recording_locations)
            if locations in recording_locations:
                j = recording_locations.index(locations)
                loc_features[j] = 1
            features['loc'].append(loc_features)

            # Extract labels 
            mm_label = get_murmur(current_patient_data)
            out_label = get_outcome(current_patient_data)
            if mm_label == 'Absent' :
                current_mm_labels = np.array([0, 1])
            elif mm_label == 'Unknown' :
                current_mm_labels = np.array([po, 1-po])
            else :
                mm_loc = get_murmur_loc(current_patient_data)
                if mm_loc == 'nan' :
                    current_mm_labels = np.array([0.9, 0.1])
                else :
                    mm_loc = mm_loc.split('+')
                    if locations in mm_loc :
                        current_mm_labels = np.array([1, 0])
                    else :
                        current_mm_labels = np.array([0.8, 0.2])

            if out_label == 'Normal' :
                current_out_labels = np.array([0, 1])
            else :
                current_out_labels = np.array([1, 0])
#                if mm_label == 'Absent' :
#                    current_out_labels = np.array([0.8, 0.2])
#                elif mm_label == 'unknown' :
#                    current_out_labels = np.array([0.85, 0.15])
#                else :
#                    current_out_labels = np.array([1, 0])
                
            mm_labels.append(current_mm_labels)
            out_labels.append(current_out_labels)

    if use_mel : 
        M, N = features['mel1'][i].shape
        for i in range(len(features['mel1'])) :
            features['mel1'][i] = features['mel1'][i].reshape(M,N,1)
        print("melspec: ", M,N)
    else :
        M, N, _ = features['mel1'][i].shape
    mel_input_shape = (M,N,1)
        
    if use_cqt :
        M, N = features['cqt1'][i].shape
        for i in range(len(features['cqt1'])) :
            features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)
        print("cqt: ", M,N)
    else :
        M, N, _ = features['cqt1'][i].shape
    cqt_input_shape = (M,N,1)

    
    if use_stft :
        M, N = features['stft1'][i].shape
        for i in range(len(features['stft1'])) :
            features['stft1'][i] = features['stft1'][i].reshape(M,N,1)
        print("stft: ", M,N)
    else :
        M, N, _ = features['stft1'][i].shape
    stft_input_shape = (M,N,1)
        
    ## RR interval    
    if use_interval:
        padded =pad_sequences(tmp_total_interval, maxlen=max_interval_len, dtype='float32', padding='post', truncating='post', value=0.0)
        
        for i in range(len(padded)):
            features['interval'].append(padded[i])
        for i in range(len(features['interval'])):
            features['interval'][i]= features['interval'][i].reshape(-1,1)
        features['interval']= np.array(features['interval'])
        
    else:
        for i in range(len(tmp_interval)):
            features['interval'].append(tmp_total_interval[i])
        for i in range(len(features['interval'])):
            features['interval'][i]= features['interval'][i].reshape(-1,1)
        features['interval']= np.array(features['interval'])

    interval_input_shape = features['interval'].shape[1:]
    M,N = interval_input_shape
    print("interval: ", M,N)
    
    ## w2v2    
    if use_wav2:
        padded =pad_sequences(tmp_wav, maxlen=maxlen1, dtype='float64', padding='post', truncating='post', value=0.0)
        padded=np.array(padded, dtype=np.float32)
        features['wav2']=padded
    
    wav2_input_shape = features['wav2'].shape[1:]
    M = wav2_input_shape
    print("wav2: ", M)    
    
        
    for k1 in features.keys() :
        features[k1] = np.array(features[k1])
    
    mm_labels = np.array(mm_labels)
    out_labels = np.array(out_labels)
    
#     return features, mm_labels, out_labels, mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape

    return features, mm_labels, out_labels, mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape,wav2_input_shape


In [29]:
if ord1 :
    features_trn, mm_lbs_trn, out_lbs_trn, mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape, wav2_input_shape = get_features_3lb_all_ord(train_folder, patient_files_trn, **params_feature)
    features_test, mm_lbs_test, out_lbs_test, _, _, _,interval_input_shape,wav2_input_shape = get_features_3lb_all_ord(test_folder, patient_files_test, **params_feature)
else :
    features_trn, mm_lbs_trn, out_lbs_trn, mel_input_shape, cqt_input_shape, stft_input_shape,interval_input_shape, wav2_input_shape = get_features_3lb_all(train_folder, patient_files_trn, **params_feature)
    features_test, mm_lbs_test, out_lbs_test, _, _, _,interval_input_shape, wav2_input_shape = get_features_3lb_all(test_folder, patient_files_test, **params_feature)



100%|█████████████████████████████████████████| 751/751 [40:49<00:00,  3.26s/it]


melspec:  120 313
interval:  192 1
wav2:  (246000,)


100%|█████████████████████████████████████████| 191/191 [08:09<00:00,  2.56s/it]


melspec:  120 313
interval:  192 1
wav2:  (246000,)


In [31]:
with open('features_trn_fix_rr_w2v2.pkl','wb') as f:
    pickle.dump(features_trn,f)
    
with open('mm_lbs_trn_fix_rr_w2v2.pkl','wb') as f:
    pickle.dump(mm_lbs_trn,f)
    
with open('out_lbs_trn_fix_rr_w2v2.pkl','wb') as f:
    pickle.dump(out_lbs_trn,f)
    
with open('mel_input_shape_fix_rr_w2v2.pkl','wb') as f:
    pickle.dump(mel_input_shape,f)

with open('cqt_input_shape_fix_rr_w2v2.pkl','wb') as f:
    pickle.dump(cqt_input_shape,f)
    
with open('stft_input_shape_fix_rr_w2v2.pkl','wb') as f:
    pickle.dump(stft_input_shape,f)
    
with open('features_test_fix_rr_w2v2.pkl','wb') as f:
    pickle.dump(features_test,f)
    
with open('mm_lbs_test_fix_rr_w2v2.pkl','wb') as f:
    pickle.dump(mm_lbs_test,f)

with open('out_lbs_test_fix_rr_w2v2.pkl','wb') as f:
    pickle.dump(out_lbs_test,f)
    
with open('interval_input_shape_fix_rr_w2v2.pkl','wb') as f:
    pickle.dump(interval_input_shape,f)
    
with open('wav2_input_shape_fix_rr_w2v2.pkl','wb') as f:
    pickle.dump(wav2_input_shape,f)

In [18]:
import pickle
with open('/Data2/hmd/hmd_sy/notebooks/features_trn_fix_rr_w2v2.pkl','rb') as f:
    features_trn = pickle.load(f)
    
with open('/Data2/hmd/hmd_sy/notebooks/mm_lbs_trn_fix_rr_w2v2.pkl','rb') as f:
    mm_lbs_trn = pickle.load(f)
    
with open('/Data2/hmd/hmd_sy/notebooks/out_lbs_trn_fix_rr_w2v2.pkl','rb') as f:
    out_lbs_trn = pickle.load(f)
    
with open('/Data2/hmd/hmd_sy/notebooks/mel_input_shape_fix_rr_w2v2.pkl','rb') as f:
    mel_input_shape = pickle.load(f)

with open('/Data2/hmd/hmd_sy/notebooks/cqt_input_shape_fix_rr_w2v2.pkl','rb') as f:
    cqt_input_shape = pickle.load(f)
    
with open('/Data2/hmd/hmd_sy/notebooks/stft_input_shape_fix_rr_w2v2.pkl','rb') as f:
    stft_input_shape = pickle.load(f)
    
with open('/Data2/hmd/hmd_sy/notebooks/features_test_fix_rr_w2v2.pkl','rb') as f:
    features_test = pickle.load(f)
    
with open('/Data2/hmd/hmd_sy/notebooks/mm_lbs_test_fix_rr_w2v2.pkl','rb') as f:
    mm_lbs_test = pickle.load(f)

with open('/Data2/hmd/hmd_sy/notebooks/out_lbs_test_fix_rr_w2v2.pkl','rb') as f:
    out_lbs_test = pickle.load(f)
    
with open('/Data2/hmd/hmd_sy/notebooks/interval_input_shape_fix_rr_w2v2.pkl','rb') as f:
    interval_input_shape = pickle.load(f)
    
with open('/Data2/hmd/hmd_sy/notebooks/wav2_input_shape_fix_rr_w2v2.pkl','rb') as f:
    wav2_input_shape = pickle.load(f)

In [19]:
def get_LCNN_o_4_dr_1(mel_input_shape, cqt_input_shape, stft_input_shape,
                      interval_input_shape,wav2_input_shape,use_wav2 = True,
                      use_mel = True, use_cqt = False, use_stft = False, ord1 = True, dp = .5, fc = False, ext = False):
        # Create a towy model.
    age = keras.Input(shape=(6,), name = 'age_cat')
    sex = keras.Input(shape=(2,), name = 'sex_cat')
    hw = keras.Input(shape=(2,), name = 'height_weight')
    preg = keras.Input(shape=(1,), name = 'is_preg')
    loc = keras.Input(shape=(5,), name = 'loc')
    interval = keras.Input(shape=interval_input_shape, name = 'interval')
    wav2 = keras.Input(shape=wav2_input_shape, name = 'wav2')
    mel1 = keras.Input(shape=mel_input_shape, name = 'mel')
    cqt1 = keras.Input(shape=cqt_input_shape, name = 'cqt')
    stft1 = keras.Input(shape=stft_input_shape, name = 'stft')
    
       
    ## age embeddig
    age1 = layers.Dense(2, activation = None)(age)

    ## sex embedding
    sex1 = layers.Dense(1, activation = None)(sex)

    ## hw embedding
    hw1 = layers.Dense(1, activation = None)(hw)

    ## loc embedding
    loc1 = layers.Dense(3, activation = None)(loc)
    
    ## interval embedding
    interval1_1 = tf.keras.layers.Conv1D(9, 3, padding="causal",activation=None,dilation_rate=1)(interval)
    interval1_2 = tf.keras.layers.Conv1D(9, 3, padding="causal",activation=None,dilation_rate=1)(interval)
    interval_mfm1 = tensorflow.keras.layers.maximum([interval1_1, interval1_2])
    interval_maxpool_1 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm1)

    interval2_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=2)(interval_maxpool_1)
    interval2_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=2)(interval_maxpool_1)
    interval_mfm2 = tensorflow.keras.layers.maximum([interval2_1, interval2_2])
    interval_maxpool_2 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm2)
    
    
    interval3_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=4)(interval_maxpool_2)
    interval3_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=4)(interval_maxpool_2)
    interval_mfm3 = tensorflow.keras.layers.maximum([interval3_1, interval3_2])
    interval_maxpool_3 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm3)
    
    interval4_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=8)(interval_maxpool_3)
    interval4_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=8)(interval_maxpool_3)
    interval_mfm4 = tensorflow.keras.layers.maximum([interval4_1, interval4_2])
    interval_maxpool_4 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm4)
    
    interval5_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=16)(interval_maxpool_4)
    interval5_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=16)(interval_maxpool_4)
    interval_mfm5 = tensorflow.keras.layers.maximum([interval5_1, interval5_1])
    interval_maxpool_5 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm5)
    
    interval6_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=32)(interval_maxpool_5)
    interval6_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=32)(interval_maxpool_5)
    interval_mfm6 = tensorflow.keras.layers.maximum([interval6_1, interval6_2])
    interval_maxpool_6 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm6)
    
    interval7_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=64)(interval_maxpool_6)
    interval7_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=64)(interval_maxpool_6)
    interval_mfm7 = tensorflow.keras.layers.maximum([interval7_1, interval7_2])
    interval_maxpool_7 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm7)
    
    interval8_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=128)(interval_maxpool_7)
    interval8_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=128)(interval_maxpool_7)
    interval_mfm8 = tensorflow.keras.layers.maximum([interval8_1, interval8_2])
    interval_maxpool_8 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm8)
    
    interval9_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=256)(interval_maxpool_8)
    interval9_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=256)(interval_maxpool_8)
    interval_mfm9 = tensorflow.keras.layers.maximum([interval9_1, interval9_2])
    interval_maxpool_9 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm9)
    
    interval10_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=512)(interval_maxpool_9)
    interval10_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=512)(interval_maxpool_9)
    interval_mfm10 = tensorflow.keras.layers.maximum([interval10_1, interval10_2])
    interval_maxpool_10 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm10)
    
    interval11_1 = tf.keras.layers.Conv1D(9, 3, padding="causal",activation=None,dilation_rate=1)(interval_maxpool_10)
    interval11_2 = tf.keras.layers.Conv1D(9, 3, padding="causal",activation=None,dilation_rate=1)(interval_maxpool_10)
    interval_mfm11 = tensorflow.keras.layers.maximum([interval11_1, interval11_2])
    interval_maxpool_11 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm11)
    
    interval12_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=2)(interval_maxpool_11)
    interval12_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=2)(interval_maxpool_11)
    interval_mfm12 = tensorflow.keras.layers.maximum([interval12_1, interval12_2])
    interval_maxpool_12 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm12)
    
    
    interval13_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=4)(interval_maxpool_12)
    interval13_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=4)(interval_maxpool_12)
    interval_mfm13 = tensorflow.keras.layers.maximum([interval13_1, interval13_2])
    interval_maxpool_13 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm13)
    
    interval14_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=8)(interval_maxpool_13)
    interval14_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=8)(interval_maxpool_13)
    interval_mfm14 = tensorflow.keras.layers.maximum([interval14_1, interval14_2])
    interval_maxpool_14 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm14)
    
    interval15_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=16)(interval_maxpool_14)
    interval15_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=16)(interval_maxpool_14)
    interval_mfm15 = tensorflow.keras.layers.maximum([interval15_1, interval15_2])
    interval_maxpool_15 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm15)
    
    interval16_1 = tf.keras.layers.Conv1D(64, 2, padding="causal",activation=None,dilation_rate=32)(interval_maxpool_15)
    interval16_2 = tf.keras.layers.Conv1D(64, 2, padding="causal",activation=None,dilation_rate=32)(interval_maxpool_15)
    interval_mfm16 = tensorflow.keras.layers.maximum([interval16_1, interval16_2])
    interval_maxpool_16 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm16)
    
    interval17_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=64)(interval_maxpool_16)
    interval17_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=64)(interval_maxpool_16)
    interval_mfm17 = tensorflow.keras.layers.maximum([interval17_1, interval17_2])
    interval_maxpool_17 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm17)
    
    interval18_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=128)(interval_maxpool_17)
    interval18_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=128)(interval_maxpool_17)
    interval_mfm18 = tensorflow.keras.layers.maximum([interval18_1, interval18_2])
    interval_maxpool_18 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm18)
    
    interval19_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=256)(interval_maxpool_18)
    interval19_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=256)(interval_maxpool_18)
    interval_mfm19 = tensorflow.keras.layers.maximum([interval19_1, interval19_2])
    interval_maxpool_19 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm19)
    
    interval20_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=512)(interval_maxpool_19)
    interval20_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=512)(interval_maxpool_19)
    interval_mfm20 = tensorflow.keras.layers.maximum([interval20_1, interval20_2])
    interval_maxpool_20 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(interval_mfm20)    
    
    interval1 = layers.GlobalAveragePooling1D()(interval_maxpool_20)
    interval1 = Dropout(dp)(interval1)
    
    ## wav2 embedding
    
    if use_wav2 :
        
        wav2vec2_layer = wav2vec2_model(wav2)
        wav2vec2_layer = tf.keras.layers.Reshape((10,768,1))(wav2vec2_layer)
        
        
        conv1_1 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(wav2vec2_layer)
        conv1_2 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(wav2vec2_layer)
        mfm2 = tensorflow.keras.layers.maximum([conv1_1, conv1_2])
        max3 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm2)

        conv4_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
        conv4_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
        mfm5 = tensorflow.keras.layers.maximum([conv4_1, conv4_2])
        batch6 = BatchNormalization(axis=3, scale=False)(mfm5)

        conv7_1 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
        conv7_2 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
        mfm8 = tensorflow.keras.layers.maximum([conv7_1, conv7_2])

        max9 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm8)
        batch10 = BatchNormalization(axis=3, scale=False)(max9)

        conv11_1 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
        conv11_2 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
        mfm12 = tensorflow.keras.layers.maximum([conv11_1, conv11_2])
        batch13 = BatchNormalization(axis=3, scale=False)(mfm12)

        conv14_1 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
        conv14_2 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
        mfm15 = tensorflow.keras.layers.maximum([conv14_1, conv14_2])

        max16 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm15)

        conv17_1 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
        conv17_2 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
        mfm18 = tensorflow.keras.layers.maximum([conv17_1, conv17_2])
        batch19 = BatchNormalization(axis=3, scale=False)(mfm18)

        conv20_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
        conv20_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
        mfm21 = tensorflow.keras.layers.maximum([conv20_1, conv20_2])
        batch22 = BatchNormalization(axis=3, scale=False)(mfm21)

        conv23_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch22)
        conv23_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch22)
        mfm24 = tensorflow.keras.layers.maximum([conv23_1, conv23_2])
        batch25 = BatchNormalization(axis=3, scale=False)(mfm24)

        conv26_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch25)
        conv26_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch25)
        mfm27 = tensorflow.keras.layers.maximum([conv26_1, conv26_2])

        max28 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(mfm27)
        wav2_1 = layers.GlobalAveragePooling2D()(max28)
        wav2_1 = Dropout(dp)(wav2_1)
        
    
#     if use_wav2:
#         wav2_reshape = tf.keras.layers.Reshape((300,-1))(wav2)
#         wav2_1_1 = tf.keras.layers.Conv1D(9, 3, padding="causal",activation=None,dilation_rate=1)(wav2_reshape)
#         wav2_1_2 = tf.keras.layers.Conv1D(9, 3, padding="causal",activation=None,dilation_rate=1)(wav2_reshape)
#         wav2_mfm1 = tensorflow.keras.layers.maximum([wav2_1_1, wav2_1_2])
#         wav2_maxpool_1 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm1)

#         wav2_2_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=2)(wav2_maxpool_1)
#         wav2_2_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=2)(wav2_maxpool_1)
#         wav2_mfm2 = tensorflow.keras.layers.maximum([wav2_2_1, wav2_2_2])
#         wav2_maxpool_2 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm2)

#         wav2_3_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=4)(wav2_maxpool_2)
#         wav2_3_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=4)(wav2_maxpool_2)
#         wav2_mfm3 = tensorflow.keras.layers.maximum([wav2_3_1, wav2_3_2])
#         wav2_maxpool_3 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm3)

#         wav2_4_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=8)(wav2_maxpool_3)
#         wav2_4_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=8)(wav2_maxpool_3)
#         wav2_mfm4 = tensorflow.keras.layers.maximum([wav2_4_1, wav2_4_2])
#         wav2_maxpool_4 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm4)

#         wav2_5_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=16)(wav2_maxpool_4)
#         wav2_5_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=16)(wav2_maxpool_4)
#         wav2_mfm5 = tensorflow.keras.layers.maximum([wav2_5_1, wav2_5_2])
#         wav2_maxpool_5 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm5)

#         wav2_6_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=32)(wav2_maxpool_5)
#         wav2_6_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=32)(wav2_maxpool_5)
#         wav2_mfm6 = tensorflow.keras.layers.maximum([wav2_6_1, wav2_6_2])
#         wav2_maxpool_6 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm6)

#         wav2_7_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=64)(wav2_maxpool_6)
#         wav2_7_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=64)(wav2_maxpool_6)
#         wav2_mfm7 = tensorflow.keras.layers.maximum([wav2_7_1, wav2_7_2])
#         wav2_maxpool_7 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm7)

#         wav2_8_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=128)(wav2_maxpool_7)
#         wav2_8_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=128)(wav2_maxpool_7)
#         wav2_mfm8 = tensorflow.keras.layers.maximum([wav2_8_1, wav2_8_2])
#         wav2_maxpool_8 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm8)

#         wav2_9_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=256)(wav2_maxpool_8)
#         wav2_9_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=256)(wav2_maxpool_8)
#         wav2_mfm9 = tensorflow.keras.layers.maximum([wav2_9_1, wav2_9_2])
#         wav2_maxpool_9 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm9)

#         wav2_10_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=512)(wav2_maxpool_9)
#         wav2_10_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=512)(wav2_maxpool_9)
#         wav2_mfm10 = tensorflow.keras.layers.maximum([wav2_10_1, wav2_10_2])
#         wav2_maxpool_10 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm10)

#         wav2_11_1 = tf.keras.layers.Conv1D(9, 3, padding="causal",activation=None,dilation_rate=1)(wav2_maxpool_10)
#         wav2_11_2 = tf.keras.layers.Conv1D(9, 3, padding="causal",activation=None,dilation_rate=1)(wav2_maxpool_10)
#         wav2_mfm11 = tensorflow.keras.layers.maximum([wav2_11_1, wav2_11_2])
#         wav2_maxpool_11 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm11)

#         wav2_12_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=2)(wav2_maxpool_11)
#         wav2_12_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=2)(wav2_maxpool_11)
#         wav2_mfm12 = tensorflow.keras.layers.maximum([wav2_12_1, wav2_12_2])
#         wav2_maxpool_12 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm12)

#         wav2_13_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=4)(wav2_maxpool_12)
#         wav2_13_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=4)(wav2_maxpool_12)
#         wav2_mfm13 = tensorflow.keras.layers.maximum([wav2_13_1, wav2_13_2])
#         wav2_maxpool_13 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm13)

#         wav2_14_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=8)(wav2_maxpool_13)
#         wav2_14_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=8)(wav2_maxpool_13)
#         wav2_mfm14 = tensorflow.keras.layers.maximum([wav2_14_1, wav2_14_2])
#         wav2_maxpool_14 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm14)

#         wav2_15_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=16)(wav2_maxpool_14)
#         wav2_15_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=16)(wav2_maxpool_14)
#         wav2_mfm15 = tensorflow.keras.layers.maximum([wav2_15_1, wav2_15_2])
#         wav2_maxpool_15 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm15)

#         wav2_16_1 = tf.keras.layers.Conv1D(64, 2, padding="causal",activation=None,dilation_rate=32)(wav2_maxpool_15)
#         wav2_16_2 = tf.keras.layers.Conv1D(64, 2, padding="causal",activation=None,dilation_rate=32)(wav2_maxpool_15)
#         wav2_mfm16 = tensorflow.keras.layers.maximum([wav2_16_1, wav2_16_2])
#         wav2_maxpool_16 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm16)

#         wav2_17_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=64)(wav2_maxpool_16)
#         wav2_17_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=64)(wav2_maxpool_16)
#         wav2_mfm17 = tensorflow.keras.layers.maximum([wav2_17_1, wav2_17_2])
#         wav2_maxpool_17 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm17)

#         wav2_18_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=128)(wav2_maxpool_17)
#         wav2_18_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=128)(wav2_maxpool_17)
#         wav2_mfm18 = tensorflow.keras.layers.maximum([wav2_18_1, wav2_18_2])
#         wav2_maxpool_18 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm18)

#         wav2_19_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=256)(wav2_maxpool_18)
#         wav2_19_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=256)(wav2_maxpool_18)
#         wav2_mfm19 = tensorflow.keras.layers.maximum([wav2_19_1, wav2_19_2])
#         wav2_maxpool_19 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm19)

#         wav2_20_1 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=512)(wav2_maxpool_19)
#         wav2_20_2 = tf.keras.layers.Conv1D(9, 2, padding="causal",activation=None,dilation_rate=512)(wav2_maxpool_19)
#         wav2_mfm20 = tensorflow.keras.layers.maximum([wav2_20_1, wav2_20_2])
#         wav2_maxpool_20 = tf.keras.layers.MaxPooling1D(pool_size=2,strides=1, padding='valid')(wav2_mfm20)    

#         wav2_1 = layers.GlobalAveragePooling1D()(wav2_maxpool_20)
#         wav2_1 = Dropout(dp)(wav2_1)
    
    
    ## mel embedding
    if use_mel :
        conv1_1 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(mel1)
        conv1_2 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(mel1)
        mfm2 = tensorflow.keras.layers.maximum([conv1_1, conv1_2])
        max3 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm2)

        conv4_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
        conv4_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
        mfm5 = tensorflow.keras.layers.maximum([conv4_1, conv4_2])
        batch6 = BatchNormalization(axis=3, scale=False)(mfm5)

        conv7_1 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
        conv7_2 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
        mfm8 = tensorflow.keras.layers.maximum([conv7_1, conv7_2])

        max9 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm8)
        batch10 = BatchNormalization(axis=3, scale=False)(max9)

        conv11_1 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
        conv11_2 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
        mfm12 = tensorflow.keras.layers.maximum([conv11_1, conv11_2])
        batch13 = BatchNormalization(axis=3, scale=False)(mfm12)

        conv14_1 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
        conv14_2 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
        mfm15 = tensorflow.keras.layers.maximum([conv14_1, conv14_2])

        max16 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm15)

        conv17_1 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
        conv17_2 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
        mfm18 = tensorflow.keras.layers.maximum([conv17_1, conv17_2])
        batch19 = BatchNormalization(axis=3, scale=False)(mfm18)

        conv20_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
        conv20_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
        mfm21 = tensorflow.keras.layers.maximum([conv20_1, conv20_2])
        batch22 = BatchNormalization(axis=3, scale=False)(mfm21)

        conv23_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch22)
        conv23_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch22)
        mfm24 = tensorflow.keras.layers.maximum([conv23_1, conv23_2])
        batch25 = BatchNormalization(axis=3, scale=False)(mfm24)

        conv26_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch25)
        conv26_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch25)
        mfm27 = tensorflow.keras.layers.maximum([conv26_1, conv26_2])

        max28 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(mfm27)
        mel2 = layers.GlobalAveragePooling2D()(max28)
        if dp :
            mel2 = Dropout(dp)(mel2)

    if use_cqt :
        ## cqt embedding
        conv1_1 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(cqt1)
        conv1_2 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(cqt1)
        mfm2 = tensorflow.keras.layers.maximum([conv1_1, conv1_2])
        max3 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm2)
        
        conv4_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
        conv4_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
        mfm5 = tensorflow.keras.layers.maximum([conv4_1, conv4_2])
        batch6 = BatchNormalization(axis=3, scale=False)(mfm5)

        conv7_1 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
        conv7_2 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
        mfm8 = tensorflow.keras.layers.maximum([conv7_1, conv7_2])

        max9 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm8)
        batch10 = BatchNormalization(axis=3, scale=False)(max9)

        conv11_1 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
        conv11_2 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
        mfm12 = tensorflow.keras.layers.maximum([conv11_1, conv11_2])
        batch13 = BatchNormalization(axis=3, scale=False)(mfm12)

        conv14_1 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
        conv14_2 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
        mfm15 = tensorflow.keras.layers.maximum([conv14_1, conv14_2])

        max16 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm15)

        conv17_1 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
        conv17_2 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
        mfm18 = tensorflow.keras.layers.maximum([conv17_1, conv17_2])
        batch19 = BatchNormalization(axis=3, scale=False)(mfm18)

        conv20_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
        conv20_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
        mfm21 = tensorflow.keras.layers.maximum([conv20_1, conv20_2])
        batch22 = BatchNormalization(axis=3, scale=False)(mfm21)

        conv23_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch22)
        conv23_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch22)
        mfm24 = tensorflow.keras.layers.maximum([conv23_1, conv23_2])
        batch25 = BatchNormalization(axis=3, scale=False)(mfm24)

        conv26_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch25)
        conv26_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch25)
        mfm27 = tensorflow.keras.layers.maximum([conv26_1, conv26_2])
        
        max28 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(mfm27)
        cqt2 = layers.GlobalAveragePooling2D()(max28)
        if dp :
            cqt2 = Dropout(dp)(cqt2)

    if use_stft :
        ## stft embedding
        conv1_1 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(stft1)
        conv1_2 = Conv2D(filters = 32, kernel_size =5, strides=(1, 1), padding='same', activation=None)(stft1)
        mfm2 = tensorflow.keras.layers.maximum([conv1_1, conv1_2])
        max3 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm2)
        
        conv4_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
        conv4_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max3)
        mfm5 = tensorflow.keras.layers.maximum([conv4_1, conv4_2])
        batch6 = BatchNormalization(axis=3, scale=False)(mfm5)
        
        conv7_1 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
        conv7_2 = Conv2D(filters = 48, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch6)
        mfm8 = tensorflow.keras.layers.maximum([conv7_1, conv7_2])
        
        max9 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm8)
        batch10 = BatchNormalization(axis=3, scale=False)(max9)
        
        conv11_1 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
        conv11_2 = Conv2D(filters = 48, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch10)
        mfm12 = tensorflow.keras.layers.maximum([conv11_1, conv11_2])
        batch13 = BatchNormalization(axis=3, scale=False)(mfm12)

        conv14_1 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
        conv14_2 = Conv2D(filters = 64, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch13)
        mfm15 = tensorflow.keras.layers.maximum([conv14_1, conv14_2])
        
        max16 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(mfm15)

        conv17_1 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
        conv17_2 = Conv2D(filters = 64, kernel_size =1, strides=(1, 1), padding='same', activation=None)(max16)
        mfm18 = tensorflow.keras.layers.maximum([conv17_1, conv17_2])
        batch19 = BatchNormalization(axis=3, scale=False)(mfm18)

        conv20_1 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
        conv20_2 = Conv2D(filters = 32, kernel_size =3, strides=(1, 1), padding='same', activation=None)(batch19)
        mfm21 = tensorflow.keras.layers.maximum([conv20_1, conv20_2])
        batch22 = BatchNormalization(axis=3, scale=False)(mfm21)

        conv23_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch22)
        conv23_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch22)
        mfm24 = tensorflow.keras.layers.maximum([conv23_1, conv23_2])
        batch25 = BatchNormalization(axis=3, scale=False)(mfm24)

        conv26_1 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch25)
        conv26_2 = Conv2D(filters = 32, kernel_size =1, strides=(1, 1), padding='same', activation=None)(batch25)
        mfm27 = tensorflow.keras.layers.maximum([conv26_1, conv26_2])
        
        max28 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(mfm27)
        stft2 = layers.GlobalAveragePooling2D()(max28)
        if dp :
            stft2 = Dropout(dp)(stft2)
    
    
    if use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2, stft2])
    if not use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([cqt2, stft2])
    if use_mel and not use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, stft2])
    if use_mel and use_cqt and not use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2])
    if not use_mel and not use_cqt and use_stft :  ## stft 만
        concat2 = stft2
    if use_mel and not use_cqt and not use_stft :  ### mel만
        concat2 = mel2
    if not use_mel and use_cqt and not use_stft :  ### cqt만
        concat2 = cqt2

    if ext :
        concat1 = layers.Concatenate()([age1, sex1, hw1, loc1, preg])
        d1 = layers.Dense(2, activation = 'relu')(concat1)
#         concat2 = layers.Concatenate()([concat2, d1, interval1])
        concat2 = layers.Concatenate()([concat2, d1, interval1, wav2_1])
        
    if fc :
        concat2 = layers.Dense(10, activation = "relu")(concat2)
        concat2 = Dropout(dp)(concat2)
        
        
    if ord1 :
        res1 = layers.Dense(2, activation = "softmax")(concat2)
    else :
        res1 = layers.Dense(3, activation = "softmax")(concat2)

        
#     res2 = layers.Dense(2, activation = "softmax")(concat2)

#     model = keras.Model(inputs = [age,sex,hw,preg,loc,interval, mel1, cqt1,stft1] , outputs = res1 )
    model = keras.Model(inputs = [age,sex,hw,preg,loc,interval, wav2, mel1,cqt1,stft1] , outputs = res1 )
    
    model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy','AUC'])
    return(model)


## Generator0.py

In [21]:
import tensorflow.keras
from tensorflow.keras import backend as K
from scipy.io import wavfile
import numpy as np
import librosa
import soundfile as sf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, maximum, add, SeparableConv2D
from sklearn.metrics import roc_curve
import os
import sys
import pickle
import soundfile
import pandas as pd
import numpy as np
import tensorflow


class Generator0():
    def __init__(self, X_train, y_train, batch_size=32, beta_param=0.2, mixup = True, lowpass = False, highpass = False, ranfilter2 = False, shuffle=True, datagen=None, chaug = False, cout = False):
        self.X_train = X_train
        self.y_train = y_train
        self.batch_size = batch_size
        self.alpha = beta_param
        self.mixup = mixup
        self.shuffle = shuffle
        self.sample_num = len(y_train)
        self.datagen = datagen

        ## ffm 
        
        self.lowpass = lowpass
        self.highpass = highpass
        self.ranfilter = ranfilter2
        self.chaug = chaug
        self.cutout = cout        


    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size * 2))

            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size * 2:(i + 1) * self.batch_size * 2]
                X, y = self.__data_generation(batch_ids)

                yield X, y

    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)

        if self.shuffle:
            np.random.shuffle(indexes)

        return indexes

    def __data_generation(self, batch_ids):
        
        
        def get_box(lambda_value, nf, nt):
            cut_rat = np.sqrt(1.0 - lambda_value)

            cut_w = int(nf * cut_rat)  # rw
            cut_h = int(nt * cut_rat)  # rh

            cut_x = int(np.random.uniform(low=0, high=nf))  # rx
            cut_y = int(np.random.uniform(low=0, high=nt))  # ry

            boundaryx1 = np.minimum(np.maximum(cut_x - cut_w // 2, 0), nf) #tf.clip_by_value(cut_x - cut_w // 2, 0, IMG_SIZE_x)
            boundaryy1 = np.minimum(np.maximum(cut_y - cut_h // 2, 0), nt) #tf.clip_by_value(cut_y - cut_h // 2, 0, IMG_SIZE_y)
            bbx2 = np.minimum(np.maximum(cut_x + cut_w // 2, 0), nf) #tf.clip_by_value(cut_x + cut_w // 2, 0, IMG_SIZE_x)
            bby2 = np.minimum(np.maximum(cut_y + cut_h // 2, 0), nt) #tf.clip_by_value(cut_y + cut_h // 2, 0, IMG_SIZE_y)

            target_h = bby2 - boundaryy1
            if target_h == 0:
                target_h += 1

            target_w = bbx2 - boundaryx1
            if target_w == 0:
                target_w += 1

            return boundaryx1, boundaryy1, target_h, target_w           
        
        
        if isinstance(self.X_train, list):
            X = []
            for X_temp in self.X_train:
                if len(X_temp.shape) == 4: 
                    _, h, w, c = X_temp.shape
                    l = np.random.beta(self.alpha, self.alpha, self.batch_size)
                    X_l = l.reshape(self.batch_size, 1, 1, 1)
                    y_l = l.reshape(self.batch_size, 1)
                elif len(X_temp.shape) == 3:
                    _, h, w = X_temp.shape
                    l = np.random.beta(self.alpha, self.alpha, self.batch_size)
                    X_l = l.reshape(self.batch_size, 1, 1)
                    y_l = l.reshape(self.batch_size, 1)
                elif len(X_temp.shape) == 2:
                    _, h = X_temp.shape
                    l = np.random.beta(self.alpha, self.alpha, self.batch_size)
                    X_l = l.reshape(self.batch_size, 1)
                    y_l = l.reshape(self.batch_size, 1)
                elif len(X_temp.shape) == 1:
                    _= X_temp.shape
                    l = np.random.beta(self.alpha, self.alpha, self.batch_size)
                    X_l = l.reshape(self.batch_size,)
                    y_l = l.reshape(self.batch_size, 1)
                
                X1 = X_temp[batch_ids[:self.batch_size]].copy()
                X2 = X_temp[batch_ids[self.batch_size:]].copy()
                
                if self.mixup :
                    Xn = X1 * X_l + X2 * (1 - X_l)
                else :
                    Xn = X1
                if len(X_temp.shape) == 4: 
                    _, h, w, c = X_temp.shape
                    if h != 1 :
                        if self.lowpass :
                            uv, lp = self.lowpass
                            dec1 = np.random.choice(2, size = self.batch_size)
                            for i in range(self.batch_size) :
                                loc1 = np.random.choice(lp, size = 1)[0]
                                Xn[i,:loc1,:,:] = 0
                        if self.highpass :
                            uv, hp = self.highpass
                            dec1 = np.random.choice(2, size = self.batch_size)
                            for i in range(self.batch_size) :
                                loc1 = np.random.choice(hp, size = 1)[0]
                                Xn[i,loc1:,:,:] = 0
                        if self.ranfilter :                
                            raniter, ranf = self.ranfilter
                            dec1 = np.random.choice(raniter, size = self.batch_size)
                            for i in range(self.batch_size) :
                                if dec1[i] > 0 :
                                    for j in range(dec1[i]) :
                                        b1 = np.random.choice(ranf, size = 1)[0]
                                        loc1 = np.random.choice(h - b1, size = 1)[0]
                                        Xn[i, loc1:(loc1 + b1 - 1), :] = 0
                        if self.chaug :
                            for i in range(self.batch_size) :
                                noiselv = np.random.uniform(low= - self.chaug, high= self.chaug)
                                Xn[i,:] += noiselv
                        if self.cutout :
                            lambda1 = np.random.beta(self.cutout, self.cutout, size = self.batch_size)   ## beta_param default : 0.7  STC페이퍼 추천은 0.6~0.8
                            for i in range(self.batch_size) :
                                boundaryx1, boundaryy1, target_h, target_w = get_box(lambda1[i], h, w)
                                Xn[i, boundaryx1:(boundaryx1+target_h), boundaryy1:(boundaryy1+target_w),: ] = 0
                
#                 if len(X_temp.shape) == 3: 
                    
#                     _, h, w = X_temp.shape
                    
#                     if h != 1 :
                        
#                         if self.lowpass :
#                             uv, lp = self.lowpass
#                             dec1 = np.random.choice(2, size = self.batch_size)
#                             for i in range(self.batch_size) :
#                                 loc1 = np.random.choice(lp, size = 1)[0]
#                                 Xn[i,:loc1,:] = 0
#                         if self.highpass :
#                             uv, hp = self.highpass
#                             dec1 = np.random.choice(2, size = self.batch_size)
#                             for i in range(self.batch_size) :
#                                 loc1 = np.random.choice(hp, size = 1)[0]
#                                 Xn[i,loc1:,:] = 0
#                         if self.ranfilter :                
#                             raniter, ranf = self.ranfilter
#                             dec1 = np.random.choice(raniter, size = self.batch_size)
#                             for i in range(self.batch_size) :
#                                 if dec1[i] > 0 :
#                                     for j in range(dec1[i]) :
#                                         b1 = np.random.choice(ranf, size = 1)[0]
#                                         loc1 = np.random.choice(h - b1, size = 1)[0]
#                                         Xn[i, loc1:(loc1 + b1 - 1), :] = 0                    
                X.append(Xn)
        else:
            if len(self.X_train.shape) == 4: 
                _, h, w, c = self.X_train.shape
                l = np.random.beta(self.alpha, self.alpha, self.batch_size)
                X_l = l.reshape(self.batch_size, 1, 1, 1)
                y_l = l.reshape(self.batch_size, 1)
            elif len(self.X_train.shape) == 3:
                _, h, w = self.X_train.shape
                l = np.random.beta(self.alpha, self.alpha, self.batch_size)
                X_l = l.reshape(self.batch_size, 1, 1)
                y_l = l.reshape(self.batch_size, 1)
            elif len(self.X_train.shape) == 2:
                _, h = self.X_train.shape
                l = np.random.beta(self.alpha, self.alpha, self.batch_size)
                X_l = l.reshape(self.batch_size, 1)
                y_l = l.reshape(self.batch_size, 1)
            elif len(self.X_train.shape) == 1:
                _= self.X_train.shape
                l = np.random.beta(self.alpha, self.alpha, self.batch_size)
                X_l = l.reshape(self.batch_size,)
                y_l = l.reshape(self.batch_size, 1)

            X1 = self.X_train[batch_ids[:self.batch_size]].copy()
            X2 = self.X_train[batch_ids[self.batch_size:]].copy()
            if self.mixup :
                Xn = X1 * X_l + X2 * (1 - X_l)
            else :
                Xn = X1

            if len(self.X_train.shape) == 4: 
                _, h, w, c = X_temp.shape
                if self.lowpass :
                    uv, lp = self.lowpass
                    dec1 = np.random.choice(2, size = self.batch_size)
                    for i in range(self.batch_size) :
                        loc1 = np.random.choice(lp, size = 1)[0]
                        Xn[i,:loc1,:,:] = 0
                if self.highpass :
                    uv, hp = self.highpass
                    dec1 = np.random.choice(2, size = self.batch_size)
                    for i in range(self.batch_size) :
                        loc1 = np.random.choice(hp, size = 1)[0]
                        Xn[i,loc1:,:,:] = 0
                if self.ranfilter :                
                    raniter, ranf = self.ranfilter
                    dec1 = np.random.choice(raniter, size = self.batch_size)
                    for i in range(self.batch_size) :
                        if dec1[i] > 0 :
                            for j in range(dec1[i]) :
                                b1 = np.random.choice(ranf, size = 1)[0]
                                loc1 = np.random.choice(h - b1, size = 1)[0]
                                Xn[i, loc1:(loc1 + b1 - 1), :] = 0
                
#                 if len(self.X_train.shape) == 3:
#                     _, h, w = X_temp.shape
                    
#                     if h != 1 :
#                         if self.lowpass :
#                             uv, lp = self.lowpass
#                             dec1 = np.random.choice(2, size = self.batch_size)
#                             for i in range(self.batch_size) :
#                                 loc1 = np.random.choice(lp, size = 1)[0]
#                                 Xn[i,:loc1,:] = 0
#                         if self.highpass :
#                             uv, hp = self.highpass
#                             dec1 = np.random.choice(2, size = self.batch_size)
#                             for i in range(self.batch_size) :
#                                 loc1 = np.random.choice(hp, size = 1)[0]
#                                 Xn[i,loc1:,:] = 0
#                         if self.ranfilter :                
#                             raniter, ranf = self.ranfilter
#                             dec1 = np.random.choice(raniter, size = self.batch_size)
#                             for i in range(self.batch_size) :
#                                 if dec1[i] > 0 :
#                                     for j in range(dec1[i]) :
#                                         b1 = np.random.choice(ranf, size = 1)[0]
#                                         loc1 = np.random.choice(h - b1, size = 1)[0]
#                                         Xn[i, loc1:(loc1 + b1 - 1), :] = 0
                
            X.append(Xn)

                
        if self.datagen:
            for i in range(self.batch_size):
                X[i] = self.datagen.random_transform(X[i])
                X[i] = self.datagen.standardize(X[i])

        if isinstance(self.y_train, list):
            y = []

            for y_train_ in self.y_train:
                y1 = y_train_[batch_ids[:self.batch_size]].copy()
                y2 = y_train_[batch_ids[self.batch_size:]].copy()
                y.append(y1 * y_l + y2 * (1 - y_l))
        else:
            y1 = self.y_train[batch_ids[:self.batch_size]].copy()
            y2 = self.y_train[batch_ids[self.batch_size:]].copy()
            y = y1 * y_l + y2 * (1 - y_l)

        return X, y
    
    

In [22]:
model1 = get_LCNN_o_4_dr_1(mel_input_shape, cqt_input_shape, stft_input_shape, interval_input_shape,wav2_input_shape,
                           use_mel = use_mel, use_cqt = use_cqt, use_stft = use_stft, ord1 = ord1, dp = dp, fc = fc, ext = ext)
model2 = get_LCNN_o_4_dr_1(mel_input_shape, cqt_input_shape, stft_input_shape, interval_input_shape,wav2_input_shape,
                           use_mel = use_mel, use_cqt = use_cqt, use_stft = use_stft, ord1 = ord1, dp = dp, fc = fc, ext = ext)

In [ ]:


n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64

if mixup :
    beta_param = .7
else :
    beta_param = 0

params = {'batch_size': batch_size,
          #          'input_shape': (100, 313, 1),
          'shuffle': True,
          'chaug': chaug,
          'beta_param': beta_param,
          'cout': cout
#              'mixup': mixup,
          #          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
          #          'highpass': [.5, [78,79,80,81,82,83,84,85]]
#              'ranfilter2' : [3, [18,19,20,21,22,23]]
          #           'dropblock' : [30, 100]
          #'device' : device
}

if mixup :
    params['mixup'] = mixup
    params['ranfilter2'] = ranfil
else :
    params['cutout'] = cout

params_no_shuffle = {'batch_size': batch_size,
                     #          'input_shape': (100, 313, 1),
                     'shuffle': False,
                     'beta_param': 0.7,
                     'mixup': False
                     #'device': device
}

if ord1 :
    class_weight = {0: mm_weight, 1: 1.}
else :
    class_weight = {0: mm_weight, 1: wunknown, 2:1.}

if mixup :
    TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
                                features_trn['interval'], features_trn['wav2'],
                          features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                         mm_lbs_trn,  ## our Y
                             **params)()
    model1.fit(TrainDGen_1,
               validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                                   features_test['preg'], features_test['loc'], 
                                   features_test['interval'],features_test['wav2'],
                                   features_test['mel1'], features_test['cqt1'], features_test['stft1']], 
                                  mm_lbs_test), 
               callbacks=[lr],
               steps_per_epoch=np.ceil(len(mm_lbs_trn)/batch_size),
               class_weight=class_weight, 
               epochs = n_epoch)

else :
    TrainGen = DataGenerator([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
                                features_trn['interval'], features_trn['wav2'],
                      features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                     mm_lbs_trn,  ## our Y
                     **params)
    model1.fit(TrainGen,
               validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                                   features_test['preg'], features_test['loc'], 
                                   features_test['interval'],features_test['wav2'],
                                   features_test['mel1'], features_test['cqt1'], features_test['stft1']], 
                                  mm_lbs_test), 
               callbacks=[lr],
               #        steps_per_epoch=np.ceil(len(mm_lbs_trn)/64),
               class_weight=class_weight, 
               epochs = n_epoch)

2022-08-12 02:09:51.686655: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-08-12 02:09:53.890874: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8100


40/40 [==============================] - 16s 302ms/step - loss: 0.9119 - accuracy: 0.7504 - auc: 0.8197 - val_loss: 10.4440 - val_accuracy: 0.2108 - val_auc: 0.2085
Epoch 2/100
40/40 [==============================] - 12s 307ms/step - loss: 0.8606 - accuracy: 0.8078 - auc: 0.8563 - val_loss: 4.9760 - val_accuracy: 0.2108 - val_auc: 0.1893
Epoch 3/100
40/40 [==============================] - 13s 325ms/step - loss: 0.8604 - accuracy: 0.8191 - auc: 0.8582 - val_loss: 0.6684 - val_accuracy: 0.5547 - val_auc: 0.6194
Epoch 4/100
40/40 [==============================] - 13s 328ms/step - loss: 0.8277 - accuracy: 0.8172 - auc: 0.8724 - val_loss: 2.0089 - val_accuracy: 0.2203 - val_auc: 0.1684
Epoch 5/100
40/40 [==============================] - 12s 315ms/step - loss: 0.8599 - accuracy: 0.8105 - auc: 0.8683 - val_loss: 0.6967 - val_accuracy: 0.4437 - val_auc: 0.4702
Epoch 6/100
40/40 [==============================] - 13s 320ms/step - loss: 0.8401 - accuracy: 0.8078 - auc: 0.8728 - val_loss: 0.7

In [ ]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
          #          'input_shape': (100, 313, 1),
          'shuffle': True,
          'chaug': 0,
          'beta_param': beta_param,
          'cout': cout,
#              'mixup': True,
          #          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
#            'highpass': [.5, [78,79,80,81,82,83,84,85]],
#              'ranfilter2' : [3, [18,19,20,21,22,23]]
        #           'dropblock' : [30, 100]
          #'device' : device
}



if mixup :
    params['mixup'] = mixup
    params['ranfilter2'] = ranfil
else :
    params['cutout'] = cout


params_no_shuffle = {'batch_size': batch_size,
                     #          'input_shape': (100, 313, 1),
                     'shuffle': False,
                     'beta_param': 0.7,
                     'mixup': False
                     #'device': device
}

class_weight = {0: oo_weight, 1: 1.}

if mixup :
    TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'],
                              features_trn['interval'], features_trn['wav2'],
                          features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                         out_lbs_trn,  ## our Y
                         **params)()

    model2.fit(TrainDGen_1,
               validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                                   features_test['preg'], features_test['loc'], 
                                   features_test['interval'],features_test['wav2'],
                                   features_test['mel1'], 
                                   features_test['cqt1'], features_test['stft1']], 
                                  out_lbs_test), 
               callbacks=[lr],
               steps_per_epoch=np.ceil(len(out_lbs_trn)/batch_size),
               class_weight=class_weight, 
               epochs = n_epoch)
else :
    TrainGen = DataGenerator([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'],
                              features_trn['interval'], features_trn['wav2'],
                      features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                     out_lbs_trn,  ## our Y
                     **params)
    model2.fit(TrainGen,
               validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                                   features_test['preg'], features_test['loc'], 
                                   features_test['interval'],features_test['wav2'],
                                   features_test['mel1'], 
                                   features_test['cqt1'], features_test['stft1']], 
                                  out_lbs_test), 
               callbacks=[lr],
               class_weight=class_weight, 
               epochs = n_epoch)
    


In [128]:
params_feature['ord1'] = ord1
params_feature['mm_mean'] = mm_mean
params_feature['dp'] = dp
params_feature['fc'] = fc
params_feature['ext'] = ext
params_feature['oo_weight'] = oo_weight
params_feature['mm_weight'] = mm_weight
params_feature['chaug'] = chaug
params_feature['cout'] = cout
params_feature['wunknown'] = wunknown
params_feature['mixup'] = mixup
params_feature['n1'] = n1

params_feature['mel_shape'] = mel_input_shape
params_feature['cqt_shape'] = cqt_input_shape
params_feature['stft_shape'] = stft_input_shape

params_feature['use_mel'] = use_mel
params_feature['use_cqt'] = use_cqt
params_feature['use_stft'] = use_stft

params_feature['interval_input_shape'] = interval_input_shape
# params_feature['raw1_input_shape'] = raw1_input_shape
params_feature['wav2_input_shape'] = wav2_input_shape
params_feature['max_interval_len'] = max_interval_len

print(params_feature)

{'samp_sec': 20, 'pre_emphasis': 0, 'hop_length': 256, 'win_length': 512, 'n_mels': 120, 'filter_scale': 1, 'n_bins': 80, 'fmin': 10, 'min_dist': 500, 'max_interval_len': 192, 'per_sec': 16000, 'use_interval': True, 'maxlen1': 246000, 'use_wav2': True, 'trim': 0, 'use_mel': True, 'use_cqt': False, 'use_stft': False, 'ord1': True, 'mm_mean': False, 'dp': 0, 'fc': False, 'ext': False, 'oo_weight': 3, 'mm_weight': 3, 'chaug': 10, 'cout': 0.8, 'wunknown': 1, 'mixup': True, 'n1': 0, 'mel_shape': (120, 313, 1), 'cqt_shape': (1, 1, 1), 'stft_shape': (1, 1, 1), 'model1': 'lcnn1_dr', 'model2': 'lcnn2_dr', 'model_fnm1': 'lcnn_fix_rr_w2v2/lcnn1_dr_model1.hdf5', 'model_fnm2': 'lcnn_fix_rr_w2v2/lcnn2_dr_model2.hdf5', 'mm_weighted_accuracy': 0.7897574123989218, 'interval_input_shape': (192, 1), 'wav2_input_shape': (246000,)}


In [129]:
# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

#    @tf.function
    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')
        


In [130]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'lcnn1_dr', m_name2 = 'lcnn2_dr', param_feature = params_feature)

run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 2)

Loading Challenge model...
Running model on Challenge data...
    1/191...


100%|█████████████████████████████████████████████| 4/4 [00:00<00:00,  4.94it/s]


    2/191...


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  4.81it/s]


    3/191...


...

    190/191...


100%|█████████████████████████████████████████████| 4/4 [00:01<00:00,  3.66it/s]


    191/191...


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  3.22it/s]


Done.


In [131]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
','.join(classes),
','.join('{:.3f}'.format(x) for x in auroc_classes),
','.join('{:.3f}'.format(x) for x in auprc_classes),
','.join('{:.3f}'.format(x) for x in f_measure_classes),
','.join('{:.3f}'.format(x) for x in accuracy_classes))

params_feature['mm_weighted_accuracy'] = weighted_accuracy

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
','.join(classes),
','.join('{:.3f}'.format(x) for x in auroc_classes),
','.join('{:.3f}'.format(x) for x in auprc_classes),
','.join('{:.3f}'.format(x) for x in f_measure_classes),
','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
+ '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string


In [132]:
## sy 내가 동일한 코드로 돌렸을 때 결과 1
print(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.762,0.613,0.526,0.801,0.768,17574.098

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.693,0.683,0.611,0.623,0.726,11426.775

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.947,0.500,0.840
AUPRC,0.883,0.073,0.884
F-measure,0.702,0.000,0.876
Accuracy,0.868,0.000,0.863

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.693,0.693
AUPRC,0.758,0.609
F-measure,0.679,0.544
Accuracy,0.776,0.462



In [123]:
## sy 내가 동일한 코드로 돌렸을 때 결과 2
print(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.760,0.610,0.545,0.822,0.790,17566.761

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.716,0.712,0.589,0.607,0.734,11453.603

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.941,0.500,0.839
AUPRC,0.860,0.073,0.897
F-measure,0.747,0.000,0.888
Accuracy,0.895,0.000,0.885

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.716,0.716
AUPRC,0.770,0.653
F-measure,0.675,0.503
Accuracy,0.796,0.409



In [133]:
label_folder = test_folder
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']

# Load and parse label and model output files.
label_files, output_files = find_challenge_files(label_folder, output_folder)
murmur_labels = load_murmurs(label_files, murmur_classes)
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
outcome_labels = load_outcomes(label_files, outcome_classes)
outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)


print(np.mean(murmur_scalar_outputs[:,0]))
print(np.mean(murmur_scalar_outputs[:,2]))
print(np.mean(outcome_scalar_outputs[:,0]))
print(np.mean(outcome_scalar_outputs[:,1]))

0.4094367159910851
0.5905632875196597
0.6565561481795386
0.34344385377087516


In [134]:
# test 3
for th1 in [0.01, 0.05, 0.1, 0.15,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8] :
    murmur_binary_outputs[:,0] = murmur_scalar_outputs[:,0] > th1
    murmur_binary_outputs[:,2] = murmur_scalar_outputs[:,2] > 1 - th1
    outcome_binary_outputs[:,0] = outcome_scalar_outputs[:,0] > th1
    outcome_binary_outputs[:,1] = outcome_scalar_outputs[:,1] > 1 - th1
    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')
    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
                 murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
                  outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)



    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
                + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string
    print("threshold: ", th1)
    print(output_string)
    print("-------------")

threshold:  0.01
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.762,0.613,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.693,0.683,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.947,0.500,0.840
AUPRC,0.883,0.073,0.884
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.693,0.693
AUPRC,0.758,0.609
F-measure,0.678,0.000
Accuracy,1.000,0.000

-------------
threshold:  0.05
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.762,0.613,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.693,0.683,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.947,0.500,0.840
AUPRC,0.883,0.073,0.884
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AU

In [114]:
# test 1
for th1 in [0.01, 0.05, 0.1, 0.15,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8] :
    murmur_binary_outputs[:,0] = murmur_scalar_outputs[:,0] > th1
    murmur_binary_outputs[:,2] = murmur_scalar_outputs[:,2] > 1 - th1
    outcome_binary_outputs[:,0] = outcome_scalar_outputs[:,0] > th1
    outcome_binary_outputs[:,1] = outcome_scalar_outputs[:,1] > 1 - th1
    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')
    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
                 murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
                  outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)



    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
                + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string
    print("threshold: ", th1)
    print(output_string)
    print("-------------")

threshold:  0.01
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.764,0.605,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.669,0.676,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.953,0.500,0.838
AUPRC,0.847,0.073,0.895
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.669,0.669
AUPRC,0.740,0.612
F-measure,0.678,0.000
Accuracy,1.000,0.000

-------------
threshold:  0.05
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.764,0.605,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.669,0.676,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.953,0.500,0.838
AUPRC,0.847,0.073,0.895
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AU

In [125]:
# test 2
for th1 in [0.01, 0.05, 0.1, 0.15,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8] :
    murmur_binary_outputs[:,0] = murmur_scalar_outputs[:,0] > th1
    murmur_binary_outputs[:,2] = murmur_scalar_outputs[:,2] > 1 - th1
    outcome_binary_outputs[:,0] = outcome_scalar_outputs[:,0] > th1
    outcome_binary_outputs[:,1] = outcome_scalar_outputs[:,1] > 1 - th1
    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')
    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
                 murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
                  outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)



    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
                + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string
    print("threshold: ", th1)
    print(output_string)
    print("-------------")

threshold:  0.01
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.760,0.610,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.716,0.712,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.941,0.500,0.839
AUPRC,0.860,0.073,0.897
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.716,0.716
AUPRC,0.770,0.653
F-measure,0.678,0.000
Accuracy,1.000,0.000

-------------
threshold:  0.05
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.760,0.610,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.716,0.712,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.941,0.500,0.839
AUPRC,0.860,0.073,0.897
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AU

In [18]:
# 교수님 출력 결과
print(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.767,0.619,0.544,0.817,0.798,17571.442

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.713,0.709,0.514,0.581,0.815,11846.100

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.958,0.500,0.842
AUPRC,0.886,0.073,0.897
F-measure,0.753,0.000,0.880
Accuracy,0.921,0.000,0.871

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.713,0.713
AUPRC,0.751,0.667
F-measure,0.695,0.333
Accuracy,0.929,0.215

